In [46]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import squarify 
import os 
import warnings
warnings.filterwarnings("ignore")

import lightgbm as lgb
#from apyori import apriori
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import zipfile
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import gc 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
zip_orders_train = "order_products__train.csv"
zip_orders_prior = "order_products__prior.csv"
zip_orders = 'orders.csv'
zip_products = 'products.csv'
zip_departments = 'departments.csv'
zip_aisles = 'aisles.csv'

archive_orders_train = zipfile.ZipFile("../instacart-market-basket-analysis/"+zip_orders_train+".zip","r")
archive_orders_prior = zipfile.ZipFile("../instacart-market-basket-analysis/"+zip_orders_prior+".zip","r")
archive_orders = zipfile.ZipFile("../instacart-market-basket-analysis/"+zip_orders+".zip","r")
archive_products = zipfile.ZipFile("../instacart-market-basket-analysis/"+zip_products+".zip","r")
archive_departments = zipfile.ZipFile("../instacart-market-basket-analysis/"+zip_departments+".zip","r")
archive_aisles = zipfile.ZipFile("../instacart-market-basket-analysis/"+zip_aisles+".zip","r")

df_order_train = pd.read_csv(archive_orders_train.open('order_products__train.csv'))
df_order_prior = pd.read_csv(archive_orders_prior.open('order_products__prior.csv'))
df_order = pd.read_csv(archive_orders.open('orders.csv'))
df_product = pd.read_csv(archive_products.open('products.csv'))
df_departments = pd.read_csv(archive_departments.open('departments.csv'))
df_aisles = pd.read_csv(archive_aisles.open('aisles.csv'))

Let's explore the dataset.

In [3]:
df_merged = pd.merge(df_product,df_aisles,on='aisle_id')
df_merged = pd.merge(df_merged,df_departments,on='department_id')

In [4]:
del [df_aisles, df_departments] #order_future, data

In [5]:
train = df_order[df_order['eval_set']=='train']
prior = df_order[df_order['eval_set']=='prior']

In [6]:
train.shape

(131209, 7)

In [7]:
train = pd.merge(train,df_order_train,on='order_id')

In [8]:
prior = pd.merge(prior,df_order_prior,on='order_id')

In [9]:
prior.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0


In [10]:
prior[prior['days_since_prior_order'].isnull()] 

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0
...,...,...,...,...,...,...,...,...,...,...
32434368,3154581,206209,prior,1,3,11,NaN,20073,9,0
32434369,3154581,206209,prior,1,3,11,NaN,13176,10,0
32434370,3154581,206209,prior,1,3,11,NaN,20842,11,0
32434371,3154581,206209,prior,1,3,11,NaN,23594,12,0


All the first orders of a customer are Nan. We can replace this value using 0

In [11]:
prior['days_since_prior_order'] = prior['days_since_prior_order'].fillna(0) 

In [12]:
#Total number of orders placed by each users
users = prior.groupby(by='user_id')['order_number'].aggregate('max').to_frame('u_num_of_orders').reset_index()

In [13]:
# Average number of products bought by the user in each purchase.
#1. First getting the total number of products in each order.
total_prd_per_order = prior.groupby(by=['user_id', 'order_id'])['product_id'].aggregate('count').to_frame('total_products_per_order').reset_index()
total_prd_per_order.head(10)

,user_id,order_id,total_products_per_order
0,1,431534,8
1,1,473747,5
2,1,550135,5
3,1,2254736,5
4,1,2295261,6
5,1,2398795,6
6,1,2539329,5
7,1,2550362,9
8,1,3108588,6
9,1,3367565,4


In [14]:
#2. Getting the average products purchased by each user
avg_products = total_prd_per_order.groupby(by=['user_id'])['total_products_per_order'].mean().to_frame('u_avg_prd').reset_index()
avg_products.head()

#deleting the total_prd_per_order dataframe
del total_prd_per_order

In [15]:
#dow of most orders placed by each user
from scipy import stats
dow = prior.groupby(by='user_id')['order_dow'].agg(lambda x: stats.mode(x)[0]).to_frame('dow_most_orders_u').reset_index()
dow.head()

,user_id,dow_most_orders_u
0,1,4
1,2,2
2,3,0
3,4,4
4,5,3


In [16]:
hod = prior.groupby(by='user_id')['order_hour_of_day'].agg(lambda x: stats.mode(x)[0]).to_frame('hod_most_orders_u').reset_index()
hod.head()

,user_id,hod_most_orders_u
0,1,7
1,2,9
2,3,16
3,4,15
4,5,18


In [17]:
#merging the user created features.

#1. merging avg_products with users
users = users.merge(avg_products, on='user_id', how='left')
#deleting avg_products
del avg_products
users.head()

,user_id,u_num_of_orders,u_avg_prd
0,1,10,5.900000
1,2,14,13.928571
2,3,12,7.333333
3,4,5,3.600000
4,5,4,9.250000


In [18]:
users = users.merge(dow, on='user_id', how='left')
#deleting dow
del dow
users.head()

,user_id,u_num_of_orders,u_avg_prd,dow_most_orders_u
0,1,10,5.900000,4
1,2,14,13.928571,2
2,3,12,7.333333,0
3,4,5,3.600000,4
4,5,4,9.250000,3


In [19]:
#3. merging hod with users
users = users.merge(hod, on='user_id', how='left')
#deleting dow
del hod
users.head()

,user_id,u_num_of_orders,u_avg_prd,dow_most_orders_u,hod_most_orders_u
0,1,10,5.900000,4,7
1,2,14,13.928571,2,9
2,3,12,7.333333,0,16
3,4,5,3.600000,4,15
4,5,4,9.250000,3,18


In [20]:
#number of time a product was purchased.
prd = prior.groupby(by='product_id')['order_id'].agg('count').to_frame('prd_count_p').reset_index()
prd.head()

,product_id,prd_count_p
0,1,1852
1,2,90
2,3,277
3,4,329
4,5,15


In [21]:
#products reorder ratio.
reorder_p = prior.groupby(by='product_id')['reordered'].agg('mean').to_frame('p_reordered_ratio').reset_index()
reorder_p.head()

,product_id,p_reordered_ratio
0,1,0.613391
1,2,0.133333
2,3,0.732852
3,4,0.446809
4,5,0.600000


In [22]:
#merging the reorder_p with prd
prd = prd.merge(reorder_p, on='product_id', how='left')
#deleting reorder_p
del reorder_p
prd.head()

,product_id,prd_count_p,p_reordered_ratio
0,1,1852,0.613391
1,2,90,0.133333
2,3,277,0.732852
3,4,329,0.446809
4,5,15,0.600000


In [23]:
#how many times a user bought the same product.
uxp = prior.groupby(by=['user_id', 'product_id'])['order_id'].agg('count').to_frame('uxp_times_bought').reset_index()
uxp.head()

,user_id,product_id,uxp_times_bought
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3


In [24]:
#reorder ratio of the user for each product.
reorder_uxp = prior.groupby(by=['user_id', 'product_id'])['reordered'].agg('mean').to_frame('uxp_reordered_ratio').reset_index()
reorder_uxp.head()

,user_id,product_id,uxp_reordered_ratio
0,1,196,0.900000
1,1,10258,0.888889
2,1,10326,0.000000
3,1,12427,0.900000
4,1,13032,0.666667


In [25]:
#merging the two dataframes into one
uxp = uxp.merge(reorder_uxp, on=['user_id', 'product_id'], how='left')
#deleting reorder_uxp
del reorder_uxp
uxp.head()

,user_id,product_id,uxp_times_bought,uxp_reordered_ratio
0,1,196,10,0.900000
1,1,10258,9,0.888889
2,1,10326,1,0.000000
3,1,12427,10,0.900000
4,1,13032,3,0.666667


In [26]:
#merging users df into uxp
df = uxp.merge(users, on='user_id', how='left')
df.head()

,user_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,hod_most_orders_u
0,1,196,10,0.900000,10,5.9,4,7
1,1,10258,9,0.888889,10,5.9,4,7
2,1,10326,1,0.000000,10,5.9,4,7
3,1,12427,10,0.900000,10,5.9,4,7
4,1,13032,3,0.666667,10,5.9,4,7


In [27]:
#merging products df into data
df = df.merge(prd, on='product_id', how='left')
df.head()

,user_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,hod_most_orders_u,prd_count_p,p_reordered_ratio
0,1,196,10,0.900000,10,5.9,4,7,35791,0.776480
1,1,10258,9,0.888889,10,5.9,4,7,1946,0.713772
2,1,10326,1,0.000000,10,5.9,4,7,5526,0.652009
3,1,12427,10,0.900000,10,5.9,4,7,6476,0.740735
4,1,13032,3,0.666667,10,5.9,4,7,3751,0.657158


In [28]:
train['days_since_prior_order'] = train['days_since_prior_order'].fillna(0) 

In [29]:
#keeping only the train eval set from the orders dataframe.
order_future = train[['user_id','product_id', 'order_id', 'reordered']]
order_future.head()

,user_id,product_id,order_id,reordered
0,1,196,1187899,1
1,1,25133,1187899,1
2,1,38928,1187899,1
3,1,26405,1187899,1
4,1,39657,1187899,1


In [30]:
gc.collect()

180

In [35]:
sampled = df.sample(frac=0.50,random_state=42)

In [36]:
sampled.head()

,user_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,hod_most_orders_u,prd_count_p,p_reordered_ratio
10589486,163834,26283,3,0.666667,15,7.333333,0,12,18906,0.583148
8128189,125629,28204,2,0.500000,11,18.272727,0,13,89632,0.711922
11763256,182247,14183,3,0.666667,4,8.000000,0,15,2794,0.614889
11247203,174134,20169,1,0.000000,13,16.307692,5,10,3054,0.527505
6993533,107989,14429,1,0.000000,51,12.431373,1,13,2761,0.228178


In [37]:
data = sampled.merge(order_future, on=['product_id', 'user_id'], how='inner')
data.head()

,user_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,hod_most_orders_u,prd_count_p,p_reordered_ratio,order_id,reordered
0,27741,5456,7,0.857143,17,7.823529,2,11,11079,0.591209,1922703,1
1,134639,4297,1,0.000000,3,11.000000,4,11,1963,0.626592,1872474,1
2,12538,33768,3,0.666667,36,9.000000,6,11,15774,0.679853,3407863,1
3,176834,46676,6,0.833333,11,13.454545,5,16,26878,0.681859,230098,1
4,59718,10644,2,0.500000,9,18.888889,6,16,12318,0.411755,319304,1


In [38]:
#deleting order_id as they are not needed for training.
data.drop(['order_id'], axis=1, inplace=True)
data.head()

,user_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,hod_most_orders_u,prd_count_p,p_reordered_ratio,reordered
0,27741,5456,7,0.857143,17,7.823529,2,11,11079,0.591209,1
1,134639,4297,1,0.000000,3,11.000000,4,11,1963,0.626592,1
2,12538,33768,3,0.666667,36,9.000000,6,11,15774,0.679853,1
3,176834,46676,6,0.833333,11,13.454545,5,16,26878,0.681859,1
4,59718,10644,2,0.500000,9,18.888889,6,16,12318,0.411755,1


In [39]:
# del [df, sampled, order_future, train]

In [40]:
data = data.merge(df_merged[['product_id', 'department_id']], on='product_id', how='left')
data.head()

,user_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,hod_most_orders_u,prd_count_p,p_reordered_ratio,reordered,department_id
0,27741,5456,7,0.857143,17,7.823529,2,11,11079,0.591209,1,20
1,134639,4297,1,0.000000,3,11.000000,4,11,1963,0.626592,1,19
2,12538,33768,3,0.666667,36,9.000000,6,11,15774,0.679853,1,4
3,176834,46676,6,0.833333,11,13.454545,5,16,26878,0.681859,1,16
4,59718,10644,2,0.500000,9,18.888889,6,16,12318,0.411755,1,13


In [41]:
gc.collect()

20

In [42]:
#setting user_id and product_id as index.
data = data.set_index(['user_id', 'product_id'])

In [43]:
data.head()

,,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,hod_most_orders_u,prd_count_p,p_reordered_ratio,reordered,department_id
user_id,product_id,,,,,,,,,,
27741,5456,7,0.857143,17,7.823529,2,11,11079,0.591209,1,20
134639,4297,1,0.000000,3,11.000000,4,11,1963,0.626592,1,19
12538,33768,3,0.666667,36,9.000000,6,11,15774,0.679853,1,4
176834,46676,6,0.833333,11,13.454545,5,16,26878,0.681859,1,16
59718,10644,2,0.500000,9,18.888889,6,16,12318,0.411755,1,13


In [44]:
y = data.reordered
X = data.drop(columns=['reordered'])

In [63]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from lightgbm import LGBMModel,LGBMClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)
d_train = lgb.Dataset(X_train, label=y_train)
d_test = lgb.Dataset(X_test, label=y_test)
watchlist = [d_test]
lgb_estimator = lgb.LGBMClassifier( 
    boosting_type="gbdt",
    objective='binary', 
    random_state=42, 
    n_estimators=100,
    metric = 'binary_logloss',
    num_leaves=127, 
    learning_rate=0.005,    
)
param_grid = {
'num_boost_round': [1000,5000],
'num_leaves': [31, 127],
}

grid = GridSearchCV(estimator=lgb_estimator, param_grid=param_grid, cv=5, verbose =10)
model = grid.fit(X=X_train, y=y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] num_boost_round=1000, num_leaves=31 .............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] . num_boost_round=1000, num_leaves=31, score=1.000, total=   0.3s
[CV] num_boost_round=1000, num_leaves=31 .............................
[LightGBM] [Warning] num_iterations is set=1000, num_boost_round=1000 will be ignored. Current value: num_iterations=1000


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] . num_boost_round=1000, num_leaves=31, score=1.000, total=   0.3s
[CV] num_boost_round=1000, num_leaves=31 .............................
[LightGBM] [Warning] num_iterations is set=1000, num_boost_round=1000 will be ignored. Current value: num_iterations=1000


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    2.2s remaining:    0.0s


[CV]  num_boost_round=1000, num_leaves=127, score=1.000, total=   0.3s
[CV] num_boost_round=1000, num_leaves=127 ............................
[LightGBM] [Warning] num_iterations is set=1000, num_boost_round=1000 will be ignored. Current value: num_iterations=1000


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    2.6s remaining:    0.0s


[CV]  num_boost_round=1000, num_leaves=127, score=1.000, total=   0.3s
[CV] num_boost_round=5000, num_leaves=31 .............................
[LightGBM] [Warning] num_iterations is set=5000, num_boost_round=5000 will be ignored. Current value: num_iterations=5000
[CV] . num_boost_round=5000, num_leaves=31, score=1.000, total=   0.3s
[CV] num_boost_round=5000, num_leaves=31 .............................
[LightGBM] [Warning] num_iterations is set=5000, num_boost_round=5000 will be ignored. Current value: num_iterations=5000
[CV] . num_boost_round=5000, num_leaves=31, score=1.000, total=   0.4s
[CV] num_boost_round=5000, num_leaves=31 .............................
[LightGBM] [Warning] num_iterations is set=5000, num_boost_round=5000 will be ignored. Current value: num_iterations=5000
[CV] . num_boost_round=5000, num_leaves=31, score=1.000, total=   0.3s
[CV] num_boost_round=5000, num_leaves=31 .............................
[LightGBM] [Warning] num_iterations is set=5000, num_boost_round=5

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    6.1s finished


In [65]:
#predicting on the testing data
y_pred = model.predict(X_test).astype('int')

#setting a threshold.
y_pred = (model.predict_proba(X_test)[:, 1] >= 0.21).astype('int')
y_pred[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [68]:
#saving the prediction as a new column in data_test
X_test['prediction'] = y_pred
X_test.head()

,,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,hod_most_orders_u,prd_count_p,p_reordered_ratio,department_id,prediction
user_id,product_id,,,,,,,,,,
70078,42585,4,0.750000,9,12.444444,6,12,29124,0.663611,20,0
131605,47144,2,0.500000,5,15.200000,1,16,34583,0.713935,16,0
89174,14272,5,0.800000,25,18.000000,5,16,182,0.478022,1,0
4123,3957,9,0.888889,16,9.937500,4,10,37973,0.788429,7,0
83804,39475,6,0.833333,17,8.352941,0,16,26443,0.706387,16,0


In [70]:
# Reset the index
final = X_test.reset_index()
# Keep only the required columns to create our submission file
final = final[['product_id', 'user_id', 'prediction']]

#gc.collect()
final.head()

,product_id,user_id,prediction
0,42585,70078,0
1,47144,131605,0
2,14272,89174,0
3,3957,4123,0
4,39475,83804,0


In [74]:
data

,,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,hod_most_orders_u,prd_count_p,p_reordered_ratio,reordered,department_id
user_id,product_id,,,,,,,,,,
27741,5456,7,0.857143,17,7.823529,2,11,11079,0.591209,1,20
134639,4297,1,0.000000,3,11.000000,4,11,1963,0.626592,1,19
12538,33768,3,0.666667,36,9.000000,6,11,15774,0.679853,1,4
176834,46676,6,0.833333,11,13.454545,5,16,26878,0.681859,1,16
59718,10644,2,0.500000,9,18.888889,6,16,12318,0.411755,1,13
...,...,...,...,...,...,...,...,...,...,...,...
115047,22993,4,0.750000,31,10.032258,4,9,4034,0.621468,1,19
18985,33184,1,0.000000,19,8.263158,6,21,2420,0.335537,1,15
160,13829,1,0.000000,12,22.083333,1,3,14664,0.538052,1,13


In [75]:
#merging our prediction with orders_test
final = final.merge(data, on='user_id', how='left')
final.head()

,product_id,user_id,prediction,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,hod_most_orders_u,prd_count_p,p_reordered_ratio,reordered,department_id
0,42585,70078,0,5,0.800000,9,12.444444,6,12,1636,0.690709,1,7
1,42585,70078,0,1,0.000000,9,12.444444,6,12,4414,0.502039,1,13
2,42585,70078,0,4,0.750000,9,12.444444,6,12,29124,0.663611,1,20
3,42585,70078,0,3,0.666667,9,12.444444,6,12,14276,0.575581,1,1
4,42585,70078,0,3,0.666667,9,12.444444,6,12,645,0.627907,1,16


In [76]:
d = dict()
for row in final.itertuples():
    if row.prediction== 1:
        try:
            d[row.order_id] += ' ' + str(row.product_id)
        except:
            d[row.order_id] = str(row.product_id)

for order in final.order_id:
    if order not in d:
        d[order] = 'None'


#We now check how the dictionary were populated (open hidden output)
#d

AttributeError: 'DataFrame' object has no attribute 'order_id'

In [ ]:
first_products = df[df['add_to_cart_order']==1]

In [ ]:
df_merged.merge(first_products,on='product_id')

In [ ]:
aisle_top = df_merged['aisle'].value_counts().nlargest(20)
aisle_top = pd.DataFrame(aisle_top.reset_index())
aisle_top.columns = ['aisle','count']
_ = sns.barplot(y = 'aisle', x='count',data=aisle_top)

In [ ]:
_ = sns.countplot(y=df_merged['department'],order = df_merged['department'].value_counts().index)

In [ ]:
a = df_merged.groupby('department')[['aisle']].sum().index.get_level_values(0).tolist() 
d = df_merged.groupby('department')[['aisle']].count().reset_index().aisle.values.tolist()
plt.figure(figsize=(12,8)) 
squarify.plot(sizes=d,label=a,alpha=0.85 )
plt.axis('off')
plt.show()

In [ ]:
gc.collect()

In [ ]:
_ = sns.countplot(y = 'order_dow',data=df)

Most of the orders are places on day 0 and day 1. This could be the weekends suggesting people order more on weekends. 

In [ ]:
_ = sns.distplot(df['order_hour_of_day'],kde=False)

Order peaks are observed in the morning from 9 am and slows down in the afternoon.

In [ ]:
data = pd.pivot_table(data=df,
                    index='order_dow',
                    values='order_number',
                    columns='order_hour_of_day')

In [ ]:
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams['font.family'] = "serif"
_ = sns.heatmap(data,cmap='coolwarm')

In [ ]:
gc.collect()

Early Sunday morning and late Saturday evenings are peak time for ordering

In [ ]:
order_by_user = pd.DataFrame(df.groupby('user_id')['order_number'].count())

In [ ]:
_ = sns.distplot(order_by_user,kde=False)

days since prior order has some missing values. Lets analyze this column

In [ ]:
df[df['days_since_prior_order'].isnull()]

For each user the nan value corresponds to their first order. For all users the first order has no record of prior days to order and so the column is set to null. We can replace this with 0 

In [ ]:
del order_by_user
del first_products
del data

In [ ]:
from  matplotlib.ticker import FuncFormatter

ax = sns.countplot(x=df['days_since_prior_order'])
ax.set_xticklabels(ax.get_xticklabels(), fontsize=7)
plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda x, _: int(x)))
plt.tight_layout()
plt.show()

The reordering has a trend to it which can be broken down into 4 groups: 
* Reorder daily: a large number of people seem to reorder on a daily basis. 
* Reorder weekly: the next visible trend is a weekly order. This could be mostly the produce and dairy items. 
* Reorder biweekly: some group of products are ordered every 15 days. 
* Reorder monthly: After that majority of reoder happends on day 30. 

We will classify reordering pattern into these 4 sub categories and explore the items purchased during each times. 

In [ ]:
# Set a default value
df['reorder_pattern'] = 'daily'
df['reorder_pattern'][df['days_since_prior_order'] <= 1] = 'daily'
df['reorder_pattern'][(df['days_since_prior_order'] > 1) & (df['days_since_prior_order'] <= 7)]  = 'weekly'
df['reorder_pattern'][(df['days_since_prior_order'] > 7) & (df['days_since_prior_order'] <= 15)]  = 'biweekly'
df['reorder_pattern'][(df['days_since_prior_order'] > 15) & (df['days_since_prior_order'] <= 30)]  = 'monthly'

In [ ]:
_ = sns.countplot(x=df['reorder_pattern'])

Most of the orders are on w weekly basis.

In [ ]:
_ = sns.distplot(df[df['reordered']==1]['product_id'],  kde=False, label='Reordered')
_ = sns.distplot(df[df['reordered']==0]['product_id'],  kde=False, label='Not reordered')


# Plot formatting
plt.legend(prop={'size': 12})
plt.show()

In [ ]:
from  matplotlib.ticker import FuncFormatter

ax = sns.countplot(x = df[df['eval_set']=='prior']['order_number'])
labels = []
for x in ax.get_yticks():
    val = round(x/1000,0)
    M = '{:}K'.format(val)
    labels.append(M)
ax = plt.gca()
ax.set_yticklabels(labels)
ax.set_xticklabels(ax.get_xticklabels(), fontsize=10)
plt.xticks(rotation='vertical')
plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda x, _: int(x)))
plt.show()

In [ ]:
df['reordered'].value_counts()/len(df['reordered'])

In [ ]:
ax = sns.countplot(x = df['reordered'])
labels = []
for x in ax.get_yticks():
    val = round(x/1000000,1)
    M = '{:}M'.format(val)
    labels.append(M)
ax = plt.gca()
ax.set_yticklabels(labels)
plt.show()

In [ ]:
df_new = df.merge(df_merged, on ='product_id')

In [ ]:
top_product = df_new.product_name.value_counts().nlargest(10)

In [ ]:
top_product = top_product.reset_index()

In [ ]:
gc.collect()

In [ ]:
_ = sns.barplot(y=top_product['index'],x=top_product['product_name'])

In [ ]:
del top_product

In [ ]:
del df_new

In [ ]:
gc.collect()

In [ ]:
df_new = df.merge(df_merged, on ='product_id')

In [ ]:
reordered = df_new[df_new['reordered']==1]

In [ ]:
del df_new

In [ ]:
top_reordered = reordered.product_name.value_counts().nlargest(10)
top_reordered = top_reordered.reset_index()

In [ ]:
_ = sns.barplot(y=top_reordered['index'],x=top_reordered['product_name'])

In [ ]:
del top_reordered
gc.collect()

In [ ]:
# popularity of the products
df_product_info = df.groupby('product_id')['reordered', 'order_id'].aggregate({'reordered': sum, 'order_id': len})

In [ ]:
df_product_info.rename(columns={'order_id': 'order_counts'}, inplace = True)
df_product_info.head()

In [ ]:
df_product_info['reorder_rate'] = df_product_info.reordered/df_product_info.order_counts

In [ ]:
high_reorder = df_product_info[df_product_info.order_counts > 200].sort_values('reorder_rate', ascending = False)
high_reorder.head()

In [ ]:
# Add to Cart - Reorder ratio:
df['add_to_cart_order_mod'] = df['add_to_cart_order'].copy()
df['add_to_cart_order_mod'].iloc[df['add_to_cart_order_mod']>60] = 60
grouped_df = df.groupby(['add_to_cart_order_mod'])['reordered'].aggregate('mean').reset_index()

plt.figure(figsize=(12,8))
sns.pointplot(grouped_df['add_to_cart_order_mod'].values, grouped_df['reordered'].values, alpha=0.8)
plt.ylabel('Reorder ratio', fontsize=12)
plt.xlabel('Add to cart order', fontsize=12)
plt.title("Add to cart order - Reorder ratio", fontsize=15)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
gc.collect()

In [ ]:
last_orders = df.groupby('user_id')['order_number'].max()


In [ ]:
last_orders.reset_index()

In [ ]:
t = pd.merge(last_orders, df,on=['user_id', 'order_number'], how='inner')[['user_id', 'product_id', 'order_id']]

In [ ]:
#del last_orders
#del grouped_df
gc.collect()

In [ ]:
t = t.groupby('order_id')['product_id'].apply(lambda x: ' '.join([str(e) for e in set(x)])).reset_index()  
t.columns = ['order_id', 'products']

In [ ]:
t.head()

In [ ]:
user_product_count = df.groupby(["user_id", 'product_id'])["order_id"].count()

In [ ]:
user_product_count.columns = ['count']
user_product_count = pd.DataFrame(user_product_count)
user_product_count.head()

In [ ]:
sorted_count = user_product_count.sort_values('count', ascending=False).sort_index(level='user_id', sort_remaining=False)

In [ ]:
gc.collect()

In [ ]:
top_k = 5
sorted_count.loc[3].index[:top_k]

In [ ]:
test_orders['products'] = df.user_id.apply(lambda x: ' '.join([str(e) for e in sorted_count.loc[x].index[:top_k]]))

In [ ]:
sub = test_orders[['order_id', 'products']].sort_values('order_id').reset_index()
sub.drop(['index'], axis = 1, inplace = True)
sub.head()

## Feature Engineering

In [ ]:
df.head()

In [ ]:
users = df.groupby(['user_id', 'product_id'])[['order_id']].count()
users.columns = ['users_product_total']
users = users.reset_index()

In [ ]:
#How frequently a customer bought a product after its first purchase ?
order_freq = df.groupby(['user_id', 'product_id'])[['order_id']].count()
order_freq.columns = ['Times_Bought_N']
total_orders = df.groupby('user_id')[['order_number']].max()
total_orders.columns = ['total_orders']
first_order_number = df.groupby(['user_id', 'product_id'])[['order_number']].min()
first_order_number.columns = ['first_order_number']
first_order_number_reset = first_order_number.reset_index()
span = pd.merge(total_orders, first_order_number_reset, on='user_id', how='right')
span['Order_Range_D'] = span.total_orders - span.first_order_number + 1
order_ratio = pd.merge(order_freq, span, on=['user_id', 'product_id'], how='left')
order_ratio['Order_Ratio_user_id_X_product_id'] = order_ratio.Times_Bought_N / order_ratio.Order_Range_D
del [order_freq, total_orders, first_order_number, span]

###########
users = users.merge(order_ratio , on=['user_id', 'product_id'], how='left')
del order_ratio
gc.collect()

In [ ]:
#Get the average, maximum & minimum order size for each customer.
order_size = df.groupby(['user_id', 'order_id'])[['product_id']].count()
order_size.columns = ['size'] 
results = order_size.groupby('user_id')[['size']].mean()
results.columns = ['order_size_avg']   
results = results.reset_index()

##########
users = users.merge(results, on=['user_id'], how='left')
del [order_size, results]
gc.collect()

In [ ]:
users = users.merge(df, on='user_id', how='left')
users.head()

In [ ]:
features = ['order_id', 'user_id', 'order_number', 'order_dow','order_hour_of_day', 'days_since_prior_order']

In [ ]:
X = df[features]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
numclass = df.product_id.unique()

In [ ]:
numclass

In [ ]:
import lightgbm as lgb
d_train = lgb.Dataset(X_train, label=y_train)
params = {}
params['learning_rate'] = 0.003
params['num_class'] = numclass
params['boosting_type'] = 'gbdt'
params['objective'] = 'multiclass'
params['metric'] = 'multi_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 10
params['min_data'] = 50
params['max_depth'] = 10
clf = lgb.train(params, d_train, 100)

In [ ]:
#Prediction
y_pred=clf.predict(X_test)
#convert into binary values

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
y_pred = np.where(y_pred>0.5,1,0)

In [ ]:
y_pred

In [ ]:
np.mean(f1_score(y_test, y_pred, average=None))

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred,y_test)